In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import ipdb
import time
import sys
import scipy.stats as stt

import chart_studio.plotly as ply
import chart_studio.tools as plytool
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.offline as plyoff
import plotly.subplots as plysub

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# my imports
sys.path.append('../src/')
from GA.GA import *
from GA.Objective import *
from Utils.Utils import *

In [ ]:
# make some data
data = stt.norm.rvs(loc=5, scale=1, size=100)

In [ ]:
# define distributions of interest
dists = ['NRM', 'GAM', 'LOG', 'EXP', 'CHI', 'STU', 'CAU', 'LPL', 'PAR']
results = dict.fromkeys(dists, None)

# get bounds
for dist in dists:
    results[dist] = [PDFParamRanges(data, dist), None, None]
    
# compute probs & loglikes
for dist in dists:
    l = ComputeLikelihood(data, results[dist][0][0], dist)
    u = ComputeLikelihood(data, results[dist][0][1], dist)
    results[dist][1] = l
    results[dist][2] = u

In [ ]:
# view results
for dist in dists:
    l = results[dist][1][0]
    u = results[dist][2][0]
    print('%s = (%0.3f, %0.3f)'%(dist, l, u))